In [1]:
import pandas as pd

In [2]:
bacteria_series = pd.read_csv("bacteria_species.csv")
bacteria_series = bacteria_series['species']
bacteria_series

0                             bacterium
1         TM7 phylum sp. oral taxon 352
2                [Eubacterium] infirmum
3         TM7 phylum sp. oral taxon 348
4                [Eubacterium] infirmum
                     ...               
5103    Arcticibacter pallidicorallinus
5104              Massilimicrobiota sp.
5105            Pontibacter ummariensis
5106               Bacillus sp. KBS0812
5107          Anaeromusa acidaminophila
Name: species, Length: 5108, dtype: object

In [3]:
import pandas as pd

# Clean bacterial names
def clean_bacteria_name(name):
    name = name.replace('[', '').replace(']', '').strip()  # Remove brackets
    if 'sp.' in name:  # Handle species abbreviations
        name = name.split('sp.')[0].strip()
    return name

bacteria_series = bacteria_series.apply(clean_bacteria_name).drop_duplicates()
bacteria_series
#bacteria_series_cleaned

0                             bacterium
1                            TM7 phylum
2                  Eubacterium infirmum
5                    bacterium 1xD42-67
7                     bacterium YEK0313
                     ...               
5102              Corynebacterium sputi
5103    Arcticibacter pallidicorallinus
5104                  Massilimicrobiota
5105            Pontibacter ummariensis
5107          Anaeromusa acidaminophila
Name: species, Length: 4682, dtype: object

In [4]:
import pandas as pd

# Load the CSV into a pandas DataFrame
df = pd.read_csv("uniprot_reference_proteomes_ids.csv", sep = ';')
df


,Proteome_ID,Tax_ID,OSCODE,SUPERREGNUM,#(1),#(2),#(3),Species Name
0,UP000521578,47692,NaN,eukaryota,14416,0,14442,Menura novaehollandiae (superb lyrebird)
1,UP000029743,1554474,ROSV2,viruses,1,0,1,Rosavirus A2 (isolate Human/Gambia/GA7403/2008)
2,UP000628017,2035344,NaN,bacteria,3942,0,3942,Amylibacter cionae
3,UP001232722,2664220,NaN,viruses,2,0,2,Tadarida brasiliensis gemykibivirus 1
4,UP000248311,1795041,NaN,bacteria,3295,0,3300,Pseudoroseicyclus aestuarii
...,...,...,...,...,...,...,...,...
24950,UP000203542,2560777,NaN,viruses,83,0,83,Spheniscid alphaherpesvirus 1
24951,UP000273303,2490946,NaN,bacteria,2387,0,2391,Actinobaculum sp. 352
24952,UP000031129,743971,NaN,bacteria,553,0,563,Mesomycoplasma flocculare ATCC 27399
24953,UP000000814,272562,CLOAB,bacteria,3847,0,3847,Clostridium acetobutylicum (strain ATCC 824 / ...


In [5]:


# Filter the DataFrame for bacteria
bacteria_df = df[df["SUPERREGNUM"] == "bacteria"]

# Create a function for loose matching
def find_proteome_id(name, species_column, proteome_column):
    for index, species in enumerate(species_column):
        if name in species:  # Check if the name is a substring of the species
            return proteome_column.iloc[index]
    return None

# Map the bacteria names in the Series to their Proteome_ID using loose matching
proteome_ids = bacteria_series.apply(
    lambda name: find_proteome_id(name, bacteria_df["Species Name"], bacteria_df["Proteome_ID"])
)

# Drop NaN values and output the result
proteome_ids = proteome_ids.dropna()
print(len(proteome_ids))  # Count of matches
print(proteome_ids)  # Matched Proteome_IDs


1976
0       UP000198817
2       UP000004504
5       UP000270441
7       UP000236625
8       UP000001410
           ...     
5098    UP000002939
5099    UP000006085
5103    UP000238034
5104    UP000195713
5105    UP000198432
Name: species, Length: 1976, dtype: object


In [ ]:
# TRY USING UNIPROT

import requests
import os
import time
from tqdm import tqdm

# Function to download proteome using the new UniProt API
def download_proteome(reference_proteome, output_dir="proteomes"):
    """
    Download the proteome of a given bacteria using the updated UniProt REST API.
    """
    url = (f'https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28proteome%3A{reference_proteome}%29%29')
    
    # Send the request to the UniProt API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Save the proteome data to a file
        filename = os.path.join(output_dir, f"{reference_proteome}_proteome.fasta")
        with open(filename, "w") as file:
            file.write(response.text)
        print(f"Proteome for {reference_proteome} downloaded successfully.")
        return True
    else:
        # Log the failure
        print(f"Failed to fetch proteome for {reference_proteome}: {response.status_code}")
        print(f"Error details: {response.text}")
        return False

# Main function to handle downloading with resumption
def download_proteomes_with_resumption(proteome_ids, output_dir="proteomes", resume_file="completed_ids.txt"):
    """
    Download proteomes with the ability to resume after interruption.
    """
    # Load completed IDs if resume file exists
    if os.path.exists(resume_file):
        with open(resume_file, "r") as file:
            completed_ids = set(line.strip() for line in file.readlines())
    else:
        completed_ids = set()

    # Open resume file in append mode to update as downloads complete
    with open(resume_file, "a") as resume_log:
        # Iterate over proteome IDs with a progress bar
        for proteome_id in tqdm(proteome_ids, desc="Downloading Proteomes"):
            if proteome_id in completed_ids:
                print(f"Skipping already downloaded proteome: {proteome_id}")
                continue
            
            # Attempt to download the proteome
            success = download_proteome(proteome_id, output_dir=output_dir)
            
            if success:
                # Log the completed ID to the resume file
                resume_log.write(proteome_id + "\n")
                resume_log.flush()  # Ensure the log is written immediately

            # Add a delay to avoid hitting rate limits
            time.sleep(0.3)

# Example usage with a list of proteome IDs
#proteome_ids = ["UP000005640", "UP000006548", "UP000000589"]  # Replace with your list of proteome IDs

download_proteomes_with_resumption(proteome_ids)



Skipping already downloaded proteome: UP000198817
Skipping already downloaded proteome: UP000004504
Skipping already downloaded proteome: UP000270441
Skipping already downloaded proteome: UP000236625
Skipping already downloaded proteome: UP000001410
Skipping already downloaded proteome: UP000008816
Skipping already downloaded proteome: UP000002230
Skipping already downloaded proteome: UP000001118
Skipping already downloaded proteome: UP000001436
Skipping already downloaded proteome: UP000001415
Skipping already downloaded proteome: UP000002438
Skipping already downloaded proteome: UP000000584
Skipping already downloaded proteome: UP000001570
Skipping already downloaded proteome: UP000001986
Skipping already downloaded proteome: UP000006735
Skipping already downloaded proteome: UP000001062
Skipping already downloaded proteome: UP000248731
Skipping already downloaded proteome: UP000000532
Proteome for UP000001735 downloaded successfully.


Proteome for UP000255382 downloaded successfully.
Proteome for UP000049472 downloaded successfully.


Proteome for UP000000809 downloaded successfully.


Proteome for UP000001417 downloaded successfully.


Proteome for UP000002198 downloaded successfully.


...

Proteome for UP000000579 downloaded successfully.


Proteome for UP000283360 downloaded successfully.


Proteome for UP000008812 downloaded successfully.


Proteome for UP000001955 downloaded successfully.


Proteome for UP000000528 downloaded successfully.
